In [1]:
from numpy import *
from pathlib import Path

In [3]:
#FP-growth(Frequent Pattern,频繁模式)可以更有效地挖掘频繁项集，但不能用于发现关联规则：
##1）构建FP树。第一次遍历会获得每个元素项的出现频率，并去掉不满足最小支持度的元素；第二次遍历构建FP树
##2）从FP树中挖掘频繁项集

In [4]:
#创建PF树的数据结构

In [28]:
class treeNode:
    def __init__(self,nameValue,numOccur,parentNode):
        self.name = nameValue
        self.count = numOccur
        self.nodeLink = None
        self.parent = parentNode
        self.children = {}
        
    def inc(self,numOccur):
        self.count += numOccur
    def disp(self,ind=1):
        print('  '*ind, self.name, ' ', self.count)
        for child in self.children.values():
            child.disp(ind+1)

In [24]:
def createTree(dataSet,minSup=1):
    headerTable = {}            #头指针表
    for trans in dataSet:
        for item in trans:
            headerTable[item] = headerTable.get(item,0) + dataSet[trans]
            
    headerTableCopy = headerTable.copy()
    for k in headerTableCopy.keys():  #remove items not meeting minSup
        if headerTable[k] < minSup: 
            del(headerTable[k])
            
    freqItemSet = set(headerTable.keys())
    if len(freqItemSet) == 0:
        return None,None
    
    for k in headerTable:
        headerTable[k] = [headerTable[k],None]
        
    retTree = treeNode('Null Set',1,None)
    for tranSet,count in dataSet.items():
        localD = {}
        for item in tranSet:
            if item in freqItemSet:
                localD[item] = headerTable[item][0]                       #添加全局频率
                
        if len(localD) > 0:
            orderedItems = [v[0] for v in sorted(localD.items(),
                                                key=lambda p:p[1], reverse=True)]  #根据全局频率对元素进行排序
            updateTree(orderedItems,retTree,headerTable,count)
    return retTree,headerTable

In [15]:
def updateTree(items,inTree,headerTable,count):
    if items[0] in inTree.children:
        inTree.children[items[0]].inc(count)
    else:                                       #不存在时，则创建一个新的节点，作为一个子节点添加到树中
        inTree.children[items[0]] = treeNode(items[0],count,inTree)
        if headerTable[items[0]][1] == None:
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1],inTree.children[items[0]])
            
    if len(items) > 1:
        updateTree(items[1::], inTree.children[items[0]],headerTable,count)

def updateHeader(nodeToTest, targetNode):         #寻找头节点的尾部插入
    while nodeToTest.nodeLink != None:
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode

In [16]:
##测试

In [17]:
def loadSimpDat():
    simpDat = [['r', 'z', 'h', 'j', 'p'],
               ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
               ['z'],
               ['r', 'x', 'n', 'o', 's'],
               ['y', 'r', 'x', 'z', 'q', 't', 'p'],
               ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]
    return simpDat

def createInitSet(dataSet):
    retDict = {}
    for trans in dataSet:
        retDict[frozenset(trans)] = 1
    return retDict

In [18]:
simpDat = loadSimpDat()
simpDat

[['r', 'z', 'h', 'j', 'p'],
 ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
 ['z'],
 ['r', 'x', 'n', 'o', 's'],
 ['y', 'r', 'x', 'z', 'q', 't', 'p'],
 ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]

In [19]:
initSet = createInitSet(simpDat)

In [20]:
initSet

{frozenset({'h', 'j', 'p', 'r', 'z'}): 1,
 frozenset({'s', 't', 'u', 'v', 'w', 'x', 'y', 'z'}): 1,
 frozenset({'z'}): 1,
 frozenset({'n', 'o', 'r', 's', 'x'}): 1,
 frozenset({'p', 'q', 'r', 't', 'x', 'y', 'z'}): 1,
 frozenset({'e', 'm', 'q', 's', 't', 'x', 'y', 'z'}): 1}

In [31]:
myFPtree,myHeaderTab = createTree(initSet,3)

In [44]:
myFPtree.disp()

   Null Set   1
     z   5
       r   1
       x   3
         s   2
           t   2
             y   2
         y   1
           t   1
             r   1
     x   1
       s   1
         r   1


In [34]:
##从FP树挖掘频繁项集
###1)从FP树中获得条件模式基
###2)利用条件模式基，构建一个条件FP树
###3)迭代重复上述步骤，直到树包含一个元素项为止

In [41]:
def ascendTree(leafNode, prefixPath): #ascends from leaf node to root
    if leafNode.parent != None:
        prefixPath.append(leafNode.name)
        ascendTree(leafNode.parent, prefixPath)
    
def findPrefixPath(basePat, treeNode): #treeNode comes from header table
    condPats = {}
    while treeNode != None:
        prefixPath = []
        ascendTree(treeNode, prefixPath)
        if len(prefixPath) > 1: 
            condPats[frozenset(prefixPath[1:])] = treeNode.count
        treeNode = treeNode.nodeLink
    return condPats

def mineTree(inTree, headerTable, minSup, preFix, freqItemList):
    bigL = [v[0] for v in sorted(headerTable.items(), key=lambda p: p[0])]#(sort header table)
    for basePat in bigL:  #start from bottom of header table
        newFreqSet = preFix.copy()
        newFreqSet.add(basePat)
        #print 'finalFrequent Item: ',newFreqSet    #append to set
        freqItemList.append(newFreqSet)
        condPattBases = findPrefixPath(basePat, headerTable[basePat][1])
        #print 'condPattBases :',basePat, condPattBases
        #2. construct cond FP-tree from cond. pattern base
        myCondTree, myHead = createTree(condPattBases, minSup)
        #print 'head from conditional tree: ', myHead
        if myHead != None: #3. mine cond. FP-tree
            print('conditional tree for: ',newFreqSet)
            myCondTree.disp(1)            
            mineTree(myCondTree, myHead, minSup, newFreqSet, freqItemList)

In [42]:
#测试PF-growth算法

In [43]:
freqItems=[]

In [45]:
mineTree(myFPtree,myHeaderTab,3,set([]),freqItems)

conditional tree for:  {'s'}
   Null Set   1
     x   3
conditional tree for:  {'t'}
   Null Set   1
     x   3
       z   3
conditional tree for:  {'z', 't'}
   Null Set   1
     x   3
conditional tree for:  {'x'}
   Null Set   1
     z   3
conditional tree for:  {'y'}
   Null Set   1
     x   3
       z   3
conditional tree for:  {'z', 'y'}
   Null Set   1
     x   3


In [46]:
freqItems

[{'r'},
 {'s'},
 {'s', 'x'},
 {'t'},
 {'t', 'x'},
 {'t', 'z'},
 {'t', 'x', 'z'},
 {'x'},
 {'x', 'z'},
 {'y'},
 {'x', 'y'},
 {'y', 'z'},
 {'x', 'y', 'z'},
 {'z'}]